<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/240203-%E5%BE%97%E5%88%B02470%E6%A0%B7%E6%9C%AC%E7%9A%842022NDRE%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97(3.10-9.10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-grn')

In [29]:
import os
import pandas as pd
import math

In [30]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/randomsample_2022_470")

In [31]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2020-01-01','2024-01-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B5']).rename('NDRE')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

2646


In [32]:
dataset = filteredSentinel.select(['NDRE','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDRE')


# 一阶谐波
harmonicIndependents_1 = ee.List(['constant', 't', 'cos1', 'sin1'])

def add_sincos_1(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

harmonicSentinel_1 = dataset.map(add_sincos_1)

harmonicTrend_1 = harmonicSentinel_1.select(harmonicIndependents_1.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_1.length(),numY=1))

harmonicTrendCoefficients_1 = harmonicTrend_1.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_1])

def fit_1(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('fitted'))

fittedHarmonic_1 = harmonicSentinel_1.map(fit_1)

In [33]:
# 5天序列NDRE
interval = 5
increment = 'day'
start = '2022-03-10'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-09-11').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDRE(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('NDRE'))

fitteds2 = s2makeup.map(add_fittedNDRE)

HANTS = fitteds2.select('NDRE')

In [34]:
# NDRE时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDRE', '1_NDRE', '2_NDRE', '3_NDRE', '4_NDRE', '5_NDRE', '6_NDRE', '7_NDRE', '8_NDRE', '9_NDRE', '10_NDRE', '11_NDRE', '12_NDRE', '13_NDRE', '14_NDRE', '15_NDRE', '16_NDRE', '17_NDRE', '18_NDRE', '19_NDRE', '20_NDRE', '21_NDRE', '22_NDRE', '23_NDRE', '24_NDRE', '25_NDRE', '26_NDRE', '27_NDRE', '28_NDRE', '29_NDRE', '30_NDRE', '31_NDRE', '32_NDRE', '33_NDRE', '34_NDRE', '35_NDRE', '36_NDRE', '37_NDRE']


In [35]:
image_agrgt_clip = image_agrgt.clip(roi)

In [36]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id'],scale=10,tileScale=8)

In [37]:
cols = ['id', '0_NDRE', '1_NDRE', '2_NDRE', '3_NDRE', '4_NDRE', '5_NDRE', '6_NDRE', '7_NDRE', '8_NDRE', '9_NDRE', '10_NDRE', '11_NDRE', '12_NDRE', '13_NDRE', '14_NDRE', '15_NDRE', '16_NDRE', '17_NDRE', '18_NDRE', '19_NDRE', '20_NDRE', '21_NDRE', '22_NDRE', '23_NDRE', '24_NDRE', '25_NDRE', '26_NDRE', '27_NDRE', '28_NDRE', '29_NDRE', '30_NDRE', '31_NDRE', '32_NDRE', '33_NDRE', '34_NDRE', '35_NDRE', '36_NDRE', '37_NDRE']
inds = range(samples_datesets.size().getInfo())

In [38]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,id,0_NDRE,1_NDRE,2_NDRE,3_NDRE,4_NDRE,5_NDRE,6_NDRE,7_NDRE,8_NDRE,...,28_NDRE,29_NDRE,30_NDRE,31_NDRE,32_NDRE,33_NDRE,34_NDRE,35_NDRE,36_NDRE,37_NDRE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(39),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,460,461,462,463,464,465,466,467,468,469
id,2045.000000,2046.000000,2047.000000,2048.000000,2049.000000,2050.000000,2051.000000,2052.000000,2053.000000,2054.000000,...,2461.000000,2462.000000,2463.000000,2464.000000,2465.000000,2466.000000,2467.000000,2468.000000,2469.000000,2470.000000
0_NDRE,0.272435,0.216328,0.223250,0.197111,0.195065,0.211916,0.169146,0.363320,0.168800,0.198052,...,0.392045,0.386750,0.363461,0.253790,0.314614,0.344401,0.249307,0.367277,0.400180,0.298352
1_NDRE,0.281065,0.221547,0.227972,0.200014,0.203793,0.219375,0.172943,0.364404,0.174425,0.197952,...,0.396144,0.389071,0.369485,0.259553,0.319005,0.349364,0.251247,0.375046,0.405892,0.304919
2_NDRE,0.289785,0.226904,0.233722,0.203425,0.213012,0.227511,0.177166,0.365960,0.180798,0.198164,...,0.400267,0.391068,0.375392,0.266094,0.323883,0.354267,0.253804,0.382824,0.411398,0.311371
3_NDRE,0.298532,0.232355,0.240457,0.207318,0.222653,0.236269,0.181784,0.367977,0.187877,0.198681,...,0.404386,0.392731,0.381142,0.273363,0.329219,0.359076,0.256959,0.390558,0.416662,0.317658


In [40]:
df.to_csv('sample_NDRE_2022_470.csv',encoding='utf-8')